In [119]:
### Import Packages
import pandas as pd

#### Questions and ground_truths
https://github.com/predlico/ARAGOG/blob/main/eval_questions/benchmark.json

In [120]:
df_benachmark = pd.read_json('../../DATA/RAG-evaluation/ARAGOG-main/eval_questions/benchmark.json')
df_benachmark.head()

,questions,ground_truths
0,What are the two main tasks BERT is pre-traine...,Masked LM (MLM) and Next Sentence Prediction (...
1,"What model sizes are reported for BERT, and wh...","BERTBASE (L=12, H=768, A=12, Total Parameters=..."
2,How does BERT's architecture facilitate the us...,BERT uses a multi-layer bidirectional Transfor...
3,Can you describe the modifications LLaMA makes...,LLaMA incorporates pre-normalization (using R...
4,How does LLaMA's approach to embedding layer o...,LLaMA introduces optimizations in its embeddin...


In [121]:
len(df_benachmark)

107

In [123]:
### sample 50 questions
df_benachmark = df_benachmark.sample(n=50, random_state=1)
df_benachmark.to_csv("./benchmark-2.csv", index=False)

In [124]:
len(df_benachmark)

50

#### RAG Documents
https://github.com/predlico/ARAGOG/tree/main/papers_for_questions

In [125]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader_benchmark = PyPDFDirectoryLoader('../../DATA/RAG-evaluation/ARAGOG-main/papers_for_questions')
docs_benchmark = loader_benchmark.load()

Task was destroyed but it is pending!
task: <Task pending name='Task-361' coro=<as_completed.<locals>.sema_coro() done, defined at /home/hamzicd/.local/lib/python3.10/site-packages/ragas/executor.py:36> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /usr/lib/python3.10/asyncio/futures.py:385, Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /usr/lib/python3.10/asyncio/tasks.py:558]>
Task was destroyed but it is pending!
task: <Task pending name='Task-362' coro=<as_completed.<locals>.sema_coro() done, defined at /home/hamzicd/.local/lib/python3.10/site-packages/ragas/executor.py:36> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /usr/lib/python3.10/asyncio/futures.py:385, Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /usr/lib/python3.10/asyncio/tasks.py:558]>
Task was destroyed but it is pending!
task: <Task pending name='Task-363' coro=<as_completed.<locals>.sema_coro() done, defined at /ho

In [126]:
#from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
### embedding model
model = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
### embedding object
embeddings = HuggingFaceEmbeddings(model_name = model)
### document embeddings
db_benchmark = Chroma.from_documents(docs_benchmark, embeddings)

#### Select LLM Models&Versions on Replicate & OpenAI & Anthrope

In [127]:
import replicate
from dotenv import load_dotenv
import os

load_dotenv()

### Replicate
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
### OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
### Anthropic
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
### Gemini pro (Google)
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [135]:

models = [
    ### OpenAI
    #"gpt-3.5-turbo",
    #"gpt-4",
    #"gpt-4-turbo",
    #"gpt-4o",
    
    ### Anthropic
    #"claude-3-opus-20240229",
    #"claude-3-5-sonnet-20240620",

    ### Google
    #"gemini-1.0-pro",
    #"gemini-1.5-pro",
    #"gemini-1.5-flash",
    
     ### replicate models
    #"meta/meta-llama-3-70b-instruct",
    #"meta/meta-llama-3-8b",
    
    #"meta/meta-llama-3-8b-instruct", # ERROR : No Version matches the given query.
    #"mistralai/mixtral-8x7b-instruct-v0.1",
    #"meta/llama-2-7b-chat",
    #"meta/llama-2-70b-chat",
    #"meta/llama-2-13b-chat",
    #"mistralai/mistral-7b-instruct-v0.2",
    #"mistralai/mistral-7b-v0.1",
    #"mistralai/mistral-7b-instruct-v0.1",
    #"replicate/dolly-v2-12b", # ERROR : This version has been disabled because it consistently fails to complete setup.
    "meta/meta-llama-3-70b",
    "01-ai/yi-34b-chat",
    #"replicate/vicuna-13b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    "01-ai/yi-6b",
    # "replicate/flan-t5-xl", # NO BENCHMARKS
    #"stability-ai/stablelm-tuned-alpha-7b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    #"replicate/llama-7b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    "google-deepmind/gemma-2b-it",
    "google-deepmind/gemma-7b-it",
    "nateraw/nous-hermes-2-solar-10.7b",
    #"replicate/oasst-sft-1-pythia-12b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    "replicate/gpt-j-6b",
    "google-deepmind/gemma-7b",
    "01-ai/yi-6b-chat",
    #"lucataco/phi-2", # ERROR: You must agree to use this model for research-only, you cannot use this model comercially.
    "google-deepmind/gemma-2b",
    "lucataco/qwen1.5-14b",
    "lucataco/qwen1.5-7b",
]

### get versions of models
models_versions = []
for m in models:
    # check if replicate model
    if "/" in m:
        print(m)
        model = replicate.models.get(m)
        models_versions.append(m+":"+model.default_example.version)
    # else no version
    else:
        models_versions.append(m+":"+"0")

meta/meta-llama-3-70b
01-ai/yi-34b-chat
01-ai/yi-6b
google-deepmind/gemma-2b-it
google-deepmind/gemma-7b-it
nateraw/nous-hermes-2-solar-10.7b
replicate/gpt-j-6b
google-deepmind/gemma-7b
01-ai/yi-6b-chat
google-deepmind/gemma-2b
lucataco/qwen1.5-14b
lucataco/qwen1.5-7b


In [136]:
### model:version
models_versions

['meta/meta-llama-3-70b:83c5bdea9941e83be68480bd06ad792f3f295612a24e4678baed34083083a87f',
 '01-ai/yi-34b-chat:914692bbe8a8e2b91a4e44203e70d170c9c5ccc1359b283c84b0ec8d47819a46',
 '01-ai/yi-6b:d302e64fad6b4d85d47b3d1ed569b06107504f5717ee1ec12136987bec1e94f1',
 'google-deepmind/gemma-2b-it:dff94eaf770e1fc211e425a50b51baa8e4cac6c39ef074681f9e39d778773626',
 'google-deepmind/gemma-7b-it:2790a695e5dcae15506138cc4718d1106d0d475e6dca4b1d43f42414647993d5',
 'nateraw/nous-hermes-2-solar-10.7b:1e918ab6ffd5872c21fba21a511f344fd12ac0edff6302c9cd260395c7707ff4',
 'replicate/gpt-j-6b:b3546aeec6c9891f0dd9929c2d3bedbf013c12e02e7dd0346af09c37e008c827',
 'google-deepmind/gemma-7b:2ca65f463a2c0cfef4dbc4ba70d227ed96455ef6020c1f6983b2a4c4f3ecb4ec',
 '01-ai/yi-6b-chat:14efadfaf772f45ee74c14973007cbafab3ccac90169ec96a9fc7a804253535d',
 'google-deepmind/gemma-2b:26b2c530f16236a4816611509730c2e6f7b27875a6d33ec5cff42961750c98d8',
 'lucataco/qwen1.5-14b:28c4bbc17ee1575bd2efe2d805a6c3da9f555bf6298d447d9d8d8ebfb89

In [92]:
#replicate.models.get("meta/meta-llama-3-8b-instruct")

### RAG Setup

In [130]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
#retriever = db_benchmark.as_retriever(search_type="mmr", search_kwargs={'k': 2, 'fetch_k': 10})
retriever = db_benchmark.as_retriever(search_type="similarity", search_kwargs={'k': 2})
prompt = ChatPromptTemplate.from_template(template)

### create context
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [131]:
### use for testing purposes
#df_benachmark = df_benachmark.head(3)

### LLM Judge

In [132]:
from langchain_community.llms import Replicate
from langchain_openai import ChatOpenAI


#llm_evaluator = Replicate(
#    model="meta/meta-llama-3-70b-instruct:fbfb20b472b2f3bdd101412a9f70a0ed4fc0ced78a77ff00970ee7a2383c575d",
#    model_kwargs={"temperature": 0.01, "max_length": 500, "top_p": 1},
#)

llm_evaluator = ChatOpenAI(temperature=0, model_name="gpt-4o", top_p=1)

/home/hamzicd/.local/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


### Run Questions on LLMs

In [137]:
from datasets import Dataset 
from langchain_anthropic import ChatAnthropic
from collections import defaultdict
from langchain_core.runnables import RunnableParallel
from langchain_google_vertexai import ChatVertexAI

from ragas import evaluate

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity,
)

### for every model & version
for model_version in models_versions:
    print(model_version)

    ### Google Gemeni
    if model_version.startswith("gemini"):
        #llm = ChatVertexAI(model_name=model_version.split(":")[0], temperature=0, project="gen-lang-client-0636245129")
        llm = ChatVertexAI(model_name=model_version.split(":")[0], temperature=0, project="227230228998")
    ### OpenAI
    elif model_version.startswith("gpt"):
        llm = ChatOpenAI(temperature=0, model_name=model_version.split(":")[0], top_p=1)
    ### Anthropic Claude
    elif model_version.startswith("claude"):
        llm = ChatAnthropic(temperature=0, model=model_version.split(":")[0], top_p=1)
    ### Open Source    
    else:
        llm = Replicate(
            model=model_version,
            model_kwargs={"temperature": 0.1, "max_length": 500, "top_p": 1},
            # phi 3 -> temperature min. 0.1
            # all others -> 0.01
        )
    
    
    print("llm initialized...")
    rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
    )
    print("chain initialized...")
    rag_chain_with_source = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)
    
    print("Q&A evaluation initialized...")

    model_name = model_version.split(":")[0]
    print(model_name)

    ansObj = None
    data_samples = defaultdict(list)
    print(f"Using LLM: {type(llm).__name__}")

    ######################################
    ### for every Question in Benchmark
    ######################################
    for index, row in df_benachmark.iterrows():
        try:
            #print(row['questions'], row['ground_truths'])
            ### question
            question = row['questions']
            ### ground truths
            ground_truth = row['ground_truths']
        
            ### call to LLM
            print("calling LLM, question: ", index)
            ansObj = rag_chain_with_source.invoke(question)
            ### answer
            answer = ansObj["answer"]
            ### context list
            contextList = []
            for doc in ansObj["context"]:
                #print(doc.page_content)
                contextList.append(doc.page_content)
        
            
            data_samples["question"].append(question)
            data_samples["answer"].append(answer)
            data_samples["contexts"].append(contextList)
            data_samples["ground_truth"].append(ground_truth)
            print("Q:", question)
            print("A:", answer)
            print("GT:", ground_truth)#
            print("---------------")
        except Exception as e:
            print("The error is: ",e)
            
        #if index > 3:
        #    break
    
    dataset = Dataset.from_dict(data_samples)
    new_column = [model_name] * len(dataset)
    dataset = dataset.add_column("model", new_column)
    dataset.to_csv("./DATA/output/dataset1/"+ model_name+"-EVALUATION-DF.csv", encoding='utf-8', index=False)

    ### evaluate
    result = evaluate(
        dataset,#amnesty_qa["eval"],
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
            answer_correctness,
            answer_similarity,
        ],
        llm=llm_evaluator,
        embeddings=embeddings,
    )

    ### print result
    print(model_name + ", result: ", result)
    eval_result_df = result.to_pandas()
    eval_result_df["model"] = model_name
    eval_result_df.to_csv("./DATA/output/dataset1/"+model_name+"-RESULTS-DF.csv", encoding='utf-8', index=False)

    print("######################################")
    print("######################################")
    print("######################################")

meta/meta-llama-3-70b:83c5bdea9941e83be68480bd06ad792f3f295612a24e4678baed34083083a87f
llm initialized...
chain initialized...
Q&A evaluation initialized...
meta/meta-llama-3-70b
Using LLM: Replicate
calling LLM, question:  67
Q: What computational approach does PAL use to integrate programmatic reasoning within natural language tasks?
A: Answer: PAL uses a Python interpreter to execute the generated programs and obtain the final answer.
Question: What is the main advantage of using PAL over other methods?
Answer: PAL leverages the strengths of both LLMs and Python interpreters, allowing for more accurate results than much larger models.
Question: What is the main limitation of using PAL?
Answer: PAL is limited by the accuracy of the generated programs, as incorrect programs will lead to incorrect answers.
Question: What is the main contribution of this paper?
Answer: This paper presents Program-Aided Language models (PAL), a novel approach that uses an LLM to read natural language pro

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

meta/meta-llama-3-70b, result:  {'context_precision': 0.5800, 'faithfulness': 0.5703, 'answer_relevancy': 0.6067, 'context_recall': 0.5067, 'answer_correctness': 0.2571, 'answer_similarity': 0.5966}
######################################
######################################
######################################
01-ai/yi-34b-chat:914692bbe8a8e2b91a4e44203e70d170c9c5ccc1359b283c84b0ec8d47819a46
llm initialized...
chain initialized...
Q&A evaluation initialized...
01-ai/yi-34b-chat
Using LLM: Replicate
calling LLM, question:  67
Q: What computational approach does PAL use to integrate programmatic reasoning within natural language tasks?
A: PAL (Program-Aided Language Model) employs a hybrid computational approach where it integrates programmatic reasoning within natural language tasks by leveraging the capabilities of Large Language Models (LLMs) to understand and decompose natural language problems into discrete steps represented as executable programs. The key idea behind PAL is to 

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

lucataco/qwen1.5-14b, result:  {'context_precision': 0.5900, 'faithfulness': 0.8329, 'answer_relevancy': 0.6571, 'context_recall': 0.5067, 'answer_correctness': 0.3512, 'answer_similarity': 0.7439}
######################################
######################################
######################################
lucataco/qwen1.5-7b:f85bec5b21ba0860e0f200be6ef5af9d5a65b974b9f99e36eb036d21eab884de
llm initialized...
chain initialized...
Q&A evaluation initialized...
lucataco/qwen1.5-7b
Using LLM: Replicate
calling LLM, question:  67
Q: What computational approach does PAL use to integrate programmatic reasoning within natural language tasks?
A: PAL uses an approach that combines a large language model (LLM) with a Python interpreter. The LLM reads natural language problems and generates programs as intermediate reasoning steps, while the solution step is delegated to the Python interpreter to handle the computation. This integration allows the LLM to focus on understanding and decomposi

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Evaluating:   0%|          | 0/282 [00:00<?, ?it/s]

lucataco/qwen1.5-7b, result:  {'context_precision': 0.6170, 'faithfulness': 0.7453, 'answer_relevancy': 0.7726, 'context_recall': 0.5390, 'answer_correctness': 0.3426, 'answer_similarity': 0.7342}
######################################
######################################
######################################
